# Preparación de datos

In [ ]:
from os import listdir
from os.path import isfile, join
MYPATH = "./datos"
wavs = sorted([f for f in listdir(MYPATH) if f[-3:]=='wav'])
print len(wavs)
print wavs[:5]

# Observaciones/mediciones

Definimos un wrapper para el script de Praat que extrae varios atributos acústicos.

**Obs.1**: Revisar el path a Praat.

**Obs.2**: En algunas veresiones de Praat debe incluirse un argumento '--run' en el llamado.

In [ ]:
import os
from subprocess import Popen, PIPE

PRAAT= '/usr/bin/praat'

def run_praat(wav, start, end, min_pitch, max_pitch):
    args= [PRAAT, 'acoustics.praat', wav, str(start),
           str(end), str(min_pitch), str(max_pitch)]
    p= Popen(args, stdout=PIPE)
    out= p.stdout.read()
    res= {}
    for line in out.split('\n'):
        line= line.strip()
        if len(line) == 0: continue
        k, v= line.split(':')
        if v == '--undefined--': v= -1
        else: v= float(v)
        res[k.lower()]= v
    return res

Algunas funciones auxiliares...

In [ ]:
def fullpath(filename): return MYPATH + '/' + filename

In [ ]:
# Given a wav filename (eg, "001m29r.wav"), return speaker's id, 
# gender (m/f), age and speech genre (s/r, for spontaneous/read).
def speaker(filename): return int(filename[0:3])
def gender(filename): return filename[3]
def age(filename): return int(filename[4:6])
def genre(filename): return filename[6]

# Testing...
for w in ["001m09r.wav", "999f99s.wav"]:
    for f in [speaker,gender,age,genre]: print f(w),
    print

Ahora ya estamos listos para medir el valor de pitch de los hombres por un lado (con rango 50-300Hz) y de las mujeres por el otro (75-500Hz).

**Obs.:** Usamos sólo las versiones espontáneas, para no tener duplicados a los hablantes. El t-test necesita independencia entre las muestras.

In [ ]:
male   = [w for w in wavs if gender(w)=='m' and genre(w)=='s']
female = [w for w in wavs if gender(w)=='f' and genre(w)=='s']

pitch_m = [run_praat(fullpath(w), 0, 0, 50, 300)['f0_mean'] for w in male]
pitch_f = [run_praat(fullpath(w), 0, 0, 75, 500)['f0_mean'] for w in female]

# Análisis exploratorio de los datos

Calculamos media y desvío estándar de cada grupo:

In [ ]:
import numpy as np
print "Hombres:", np.mean(pitch_m), np.std(pitch_m)
print "Mujeres:", np.mean(pitch_f), np.std(pitch_f)

Generamos **histogramas** para analizar las distribuciones de las observaciones:

In [ ]:
import matplotlib.pyplot as plt
plt.hist(pitch_m, facecolor='blue', alpha=0.5)
plt.hist(pitch_f, facecolor='green', alpha=0.5)
plt.show()

Ahora generamos **box plots**:

In [ ]:
import matplotlib.pyplot as plt
plt.boxplot([pitch_m,pitch_f])
plt.show()

# Análisis estadístico

Nuestra hipótesis nula ($H_0$) es que los dos grupos (hombres y mujeres) son iguales en términos de nivel tonal. La hipótesis alternativa ($H_1$) es que los hombres tienen voz más grave que las mujeres, cosa que el análisis de arriba parece sugerir.

Vamos a correr entonces un **$t$-test de Student**, con una sola cola (one-tailed). Si $H_1$ dijera "distinto" en lugar de "más grave que", entonces correríamos un test con dos colas (two-tailed).

El $p$-valor arrojado por la función ttest_ind es two-tailed. Para pasarlo a one-tailed sólo necesitamos dividirlo por 2.

In [ ]:
from scipy import stats
stats.ttest_ind(pitch_m, pitch_f).pvalue / 2

# Conclusiones
El $p$-valor resultante es bajísimo, lo cual constituye sólida evidencia empírica de que deberíamos rechazar $H_0$ y quedarnos con $H_1$ (los hombres en general tienen la voz más grave que las mujeres).